## Installation

In [16]:
!python -m pip install datasets transformers tensorboard tensorflow ipywidgets

In [2]:
from huggingface_hub import notebook_login

notebook_login()


In [17]:
import tensorflow as tf
print(tf.config.experimental.list_physical_devices('GPU'))

model_id = "google/vit-base-patch16-224-in21k"

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [18]:
import os
import datasets

def create_image_folder_dataset(root_path):
  """creates `Dataset` from image folder structure"""
  
  # get class names by folders names
  _CLASS_NAMES= os.listdir(root_path)
  # defines `datasets` features`
  features=datasets.Features({
                      "img": datasets.Image(),
                      "label": datasets.features.ClassLabel(names=_CLASS_NAMES),
                  })
  # temp list holding datapoints for creation
  img_data_files=[]
  label_data_files=[]
  # load images into list for creation
  for img_class in os.listdir(root_path):
    for img in os.listdir(os.path.join(root_path,img_class)):
      path_=os.path.join(root_path,img_class,img)
      img_data_files.append(path_)
      label_data_files.append(img_class)
  # create dataset
  ds = datasets.Dataset.from_dict({"img":img_data_files,"label":label_data_files},features=features)
  return ds


In [19]:
raw_dataset = create_image_folder_dataset("images")

In [20]:
img_class_labels = raw_dataset.features["label"].names
img_class_labels

['Esbjerg Kommune',
 'Horsens Kommune',
 'Slagelse Kommune',
 'Roskilde Kommune',
 'Randers Kommune',
 'Gentofte Kommune',
 'Kolding Kommune',
 'Gladsaxe Kommune',
 'Silkeborg Kommune',
 'Kalundborg Kommune']

In [21]:
from transformers import ViTFeatureExtractor
from tensorflow import keras 
from tensorflow.keras import layers


feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)

# learn more about data augmentation here: https://www.tensorflow.org/tutorials/images/data_augmentation
data_augmentation = keras.Sequential(
    [
        layers.Resizing(feature_extractor.size, feature_extractor.size),
        layers.Rescaling(1./255)
    ],
    name="data_augmentation",
)
# use keras image data augementation processing
def augmentation(examples):
    # print(examples["img"])
    examples["pixel_values"] = [data_augmentation(image) for image in examples["img"]]
    return examples


# basic processing (only resizing)
def process(examples):
    examples.update(feature_extractor(examples['img'], ))
    return examples

# we are also renaming our label col to labels to use `.to_tf_dataset` later
raw_dataset = raw_dataset.rename_column("label", "labels")



/opt/homebrew/anaconda3/envs/tf/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [22]:
processed_dataset = raw_dataset.map(process, batched=True)
processed_dataset

Map:   0%|          | 0/2802 [00:00<?, ? examples/s]

Dataset({
    features: ['img', 'labels', 'pixel_values'],
    num_rows: 2802
})

In [23]:
# test size will be 15% of train dataset
test_size=.15

processed_dataset = processed_dataset.shuffle().train_test_split(test_size=test_size)
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['img', 'labels', 'pixel_values'],
        num_rows: 2381
    })
    test: Dataset({
        features: ['img', 'labels', 'pixel_values'],
        num_rows: 421
    })
})

In [24]:
from huggingface_hub import HfFolder
import tensorflow as tf

id2label = {str(i): label for i, label in enumerate(img_class_labels)}
label2id = {v: k for k, v in id2label.items()}

num_train_epochs = 5
train_batch_size = 32
eval_batch_size = 32
learning_rate = 6e-5
weight_decay_rate=0.01
num_warmup_steps=0
output_dir=model_id.split("/")[1]
hub_token = HfFolder.get_token()
hub_model_id = f'{model_id.split("/")[1]}-street-view'


In [25]:
from transformers import DefaultDataCollator

# Data collator that will dynamically pad the inputs received, as well as the labels.
data_collator = DefaultDataCollator(return_tensors="tf")

# converting our train dataset to tf.data.Dataset
tf_train_dataset = processed_dataset["train"].to_tf_dataset(
   columns=['pixel_values'],
   label_cols=["labels"],
   shuffle=True,
   batch_size=train_batch_size,
   collate_fn=data_collator)

# converting our test dataset to tf.data.Dataset
tf_eval_dataset = processed_dataset["test"].to_tf_dataset(
   columns=['pixel_values'],
   label_cols=["labels"],
   shuffle=True,
   batch_size=eval_batch_size,
   collate_fn=data_collator)

/opt/homebrew/anaconda3/envs/tf/lib/python3.9/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [26]:
from transformers import TFViTForImageClassification, create_optimizer
import tensorflow as tf

# create optimizer wight weigh decay
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps,
)

# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)

# define loss
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# define metrics 
metrics=[
    tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(3, name="top-3-accuracy"),
]

# compile model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/kernel:0', 'vit/pooler/dense/bias:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
import os
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard as TensorboardCallback, EarlyStopping

callbacks = []
callbacks.append(TensorboardCallback(log_dir=os.path.join(output_dir, "logs")))
callbacks.append(EarlyStopping(monitor="val_accuracy",patience=1))
callbacks.append(PushToHubCallback(
    output_dir,
    hub_model_id=hub_model_id,
    hub_token=hub_token,
))



Cloning https://huggingface.co/csiztom/vit-base-patch16-224-in21k-street-view into local empty directory.


Download file tf_model.h5:   0%|          | 16.0k/328M [00:00<?, ?B/s]

Download file logs/validation/events.out.tfevents.1698832044.d55519.64961.1.v2: 100%|##########| 2.00k/2.00k […

Clean file logs/validation/events.out.tfevents.1698832044.d55519.64961.1.v2:  50%|####9     | 1.00k/2.00k [00:…

Download file logs/train/events.out.tfevents.1698831820.d55519.64961.0.v2:   1%|          | 17.5k/2.84M [00:00…

Clean file logs/train/events.out.tfevents.1698831820.d55519.64961.0.v2:   0%|          | 1.00k/2.84M [00:00<?,…

Clean file tf_model.h5:   0%|          | 1.00k/328M [00:00<?, ?B/s]

In [28]:
train_results = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=callbacks,
    epochs=num_train_epochs,
)

Epoch 1/5
75/75 [==============================] - 271s 3s/step - loss: 2.2491 - accuracy: 0.1756 - top-3-accuracy: 0.4351 - val_loss: 2.1890 - val_accuracy: 0.1853 - val_top-3-accuracy: 0.4917
Epoch 2/5
75/75 [==============================] - 237s 3s/step - loss: 1.9960 - accuracy: 0.3809 - top-3-accuracy: 0.7043 - val_loss: 2.0343 - val_accuracy: 0.2779 - val_top-3-accuracy: 0.5962
Epoch 3/5
75/75 [==============================] - 235s 3s/step - loss: 1.6775 - accuracy: 0.5804 - top-3-accuracy: 0.8681 - val_loss: 1.9208 - val_accuracy: 0.3420 - val_top-3-accuracy: 0.6580
Epoch 4/5
75/75 [==============================] - 233s 3s/step - loss: 1.3661 - accuracy: 0.7648 - top-3-accuracy: 0.9496 - val_loss: 1.8553 - val_accuracy: 0.3753 - val_top-3-accuracy: 0.6698
Epoch 5/5
75/75 [==============================] - 393s 5s/step - loss: 1.1534 - accuracy: 0.8748 - top-3-accuracy: 0.9786 - val_loss: 1.8468 - val_accuracy: 0.3610 - val_top-3-accuracy: 0.6722
